In [6]:
import os
import boto3
import json
import tempfile

# SET PARAMETERS TO ACCESS S3 BACKEND
s3Path = 'PATH IN S3 BUCKET WHERE TRAINING RECORDS ARE STORED'
s3Destination = 'PATH IN S3 BUCKET WHERE YOU WANT TO STORE THE TRAINED MODEL - INCLUDING MODEL NAME'
s3endpointUrl = 'ENDPOINT URL TO ACCESS S3 BACKEND'
s3objectStoreLocation = 'S3 BUCKET NAME'
s3accessKey = 'S3 ACCESS KEY'
s3secretKey = 'S3 SECRET KEY'

# CREATE TEMP DIRECTORY TO DOWNLOAD DATA
tmpdir = str(tempfile.mkdtemp())
SOURCE_DIR = tmpdir + '/' + 'bots/images/'
if not os.path.exists(SOURCE_DIR):
   os.makedirs(SOURCE_DIR)
if os.path.exists(tmpdir + "/" + "bots/images/data.jsonl"):
   os.remove(tmpdir + "/" + "bots/images/data.jsonl")
fname = tmpdir + "/" + 'bots/images/data.jsonl'
print(fname)

# CREATE CONNECTION TO S3 BACKEND
session = boto3.Session(aws_access_key_id=s3accessKey,
                        aws_secret_access_key=s3secretKey)
s3 = session.resource('s3', endpoint_url=s3endpointUrl, verify=False)

# DOWNLOAD TRAINING DATA
objects = []
bucket = s3.Bucket(name=s3objectStoreLocation)
FilesNotFound = True
for obj in bucket.objects.filter(Prefix=s3Path):
   objects.append(obj.key)

for key in objects:
    obj = s3.Object(s3objectStoreLocation, key)
    contents = obj.get()['Body'].read().decode('utf-8')
    if contents:
        jcontents = json.loads(contents)
        with open(fname, 'a') as outfile:
            json.dump(jcontents, outfile)
            outfile.write('\n')

# TRAIN MODEL
print("Start Training ...")
cmd = "python bots/learn-tests --dry " + fname
os.system(cmd)
print("Training complete!")

# SEARCH FOR TRAINED MODEL IN TEMP DIRECTORY
cmd = 'find ' + tmpdir + '/bots/images/ -name tests-learn*.model -printf "%f"'
model = os.popen(cmd).read()
print("Found model : " + model)

# UPLOAD TRAINED MODEL TO DESTINATION FOLDER IN S3 BACKEND
print("Uploading model to: " + s3Destination)
s3.meta.client.upload_file(tmpdir + "/bots/images/" + model, 
                           s3objectStoreLocation, s3Destination)
print("Model upload to S3 backend Complete!")


/tmp/tmpu43uu3ff/bots/images/data.jsonl
Start Training ...
Training complete!
Found model : tests-learn-5-3.model
Uploading model to: cchase/flake-old/models/sample.model
Model upload to S3 backend Complete!
